#loading package

In [1]:
# !pip install causalnex

In [2]:
# !pip install "causalnex[all]"

In [6]:
!apt install libgraphviz-dev
!pip install pygraphviz
!pip install dowhy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgraphviz-dev is already the newest version (2.42.2-3build2).
0 upgraded, 0 newly installed, 0 to remove and 46 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# !pip install git+https://github.com/microsoft/dowhy.git
# !apt install libgraphviz-dev
# !pip install pygraphviz

In [ ]:


# !pip install git+https://github.com/uber/causalml.git

In [ ]:
# !pip install econml

In [ ]:

#!pip install xgboost
#!pip install lightgbm
#!pip install shap

In [ ]:
# !pip uninstall numpy --y
# !pip install numpy

#Data Pre-Proceessing

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

RELOAD_PICKLES = False
datathon_directory = 'drive/MyDrive/datathon23'
pickles_directory = f'{datathon_directory}/temp_pickles/'
sql_script_directory = f'{datathon_directory}/sql_scripts/'
output_directory = f'{datathon_directory}/outputs/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from IPython.display import clear_output
from tqdm.notebook import tqdm  as tqdm



In [3]:
import dowhy as dw

In [4]:
##plt.style.use('fivethirtyeight')

In [5]:
df_all = pd.read_csv(f"{output_directory}/dfall_sfr.csv").set_index('admissionid')

In [6]:
df_all.columns

Index(['measuredat2', 'is_iv', 'iv_model', 'iv_mode', 'entubated', 'tube',
       'tracheostoma', 'a2_resp_rate', 'tidal', 'minute_vol', 'peep', 'peak',
       'insp_p', 'map', 'sbp', 'dbp', 'plt', 'insp_time', 'spo2',
       'a2_heart_rate', 'a2_temperature', 'a2_mean_abp', 'PaO2', 'PaCO2',
       'FiO2', 'AaDO2', 'a2_pH', 'a2_sodium', 'a2_potassium',
       'a2_creatinine_mg_per_dl', 'a2_ht', 'is_surgical', 'is_medical',
       'is_cardiothoracic_surgery', 'is_respiratory_failure',
       'is_neurosurgery', 'is_gastrointestinal_surgery', 'is_cardiac_arrest',
       'is_vascular_surgery', 'is_trauma', 'is_neuro', 'is_cardio',
       'is_sepsis', 'is_infection', 'is_shock', 'is_invasive_ventilation',
       'hr', 'mp', 'sfr'],
      dtype='object')

In [7]:
# df_mp2 = pd.read_csv(f"{output_directory}/df_mp_arif.csv")

In [8]:
dotgraph = "digraph { map -> vfd28; sbp -> vfd28; dbp -> vfd28; plt -> vfd28; a2_heart_rate -> vfd28; a2_temperature -> vfd28; a2_mean_abp -> vfd28; a2_pH -> vfd28; a2_sodium -> vfd28; a2_potassium -> vfd28; a2_creatinine_mg_per_dl -> vfd28; a2_ht -> vfd28; mp -> vfd28; a2_resp_rate -> mp; tidal -> mp; minute_vol -> mp; peep -> mp; insp_p -> mp }"
dotgraph = dotgraph.replace("vfd28", 'sfr')
dotgraph

'digraph { map -> sfr; sbp -> sfr; dbp -> sfr; plt -> sfr; a2_heart_rate -> sfr; a2_temperature -> sfr; a2_mean_abp -> sfr; a2_pH -> sfr; a2_sodium -> sfr; a2_potassium -> sfr; a2_creatinine_mg_per_dl -> sfr; a2_ht -> sfr; mp -> sfr; a2_resp_rate -> mp; tidal -> mp; minute_vol -> mp; peep -> mp; insp_p -> mp }'

In [9]:
dag_columns = list(set([x for x in re.split(r"[->; }]|\{", dotgraph.replace('digraph',''),) if x]))
dag_columns
# df_all = df_all[dag_columns]

['a2_sodium',
 'a2_potassium',
 'plt',
 'peep',
 'sfr',
 'a2_heart_rate',
 'minute_vol',
 'a2_temperature',
 'a2_ht',
 'dbp',
 'a2_mean_abp',
 'sbp',
 'tidal',
 'insp_p',
 'a2_resp_rate',
 'map',
 'a2_creatinine_mg_per_dl',
 'mp',
 'a2_pH']

In [10]:
mp_cols = ['peep', 'a2_resp_rate', 'insp_p', 'tidal']

In [11]:
categories = ["is_surgical","is_medical"]

mp_categories = list(range(1,41))[::-1]
hr_categories = list(range(1,25))

df_all_copy = df_all[dag_columns + ['hr'] + categories].dropna().copy()

print(categories)
print(hr_categories)
print(mp_categories)

['is_surgical', 'is_medical']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1]


In [12]:
try:
  result_df = pd.read_excel(f"{output_directory}/estimates_q10.xlsx", index_col=[0,1,2,])
except:
  result_df = pd.DataFrame([], columns=['cat','hr','mp','value','ci_lower', 'ci_upper'])
  result_df = result_df.set_index(['cat','hr','mp',])

# display(result_df)
for cat in categories:
  df_col =  df_all_copy[df_all_copy[cat]]
  for mp in tqdm(mp_categories):
    df_mp = df_col.copy()
    df_mp['mp'] = df_mp['mp'] >= mp
    for hr in hr_categories:
      idx = (cat,hr,mp)
      if idx not in result_df.index:
        df_hr = df_mp[df_mp['hr']==hr].copy()
        model = dw.CausalModel(data = df_hr,treatment = "mp",outcome = "sfr", graph = dotgraph)
        id_effect = model.identify_effect(proceed_when_unidentifiable = True)
        estimate = model.estimate_effect(identified_estimand = id_effect, method_name = "backdoor.linear_regression", confidence_intervals = True)

        # ci = estimate.get_confidence_intervals()
        ci = [np.nan, np.nan]
        # results[(cat,hr,mp)] = {'v':estimate.value, 'ci': estimate.get_confidence_intervals()}
        result_df.loc[(cat,hr,mp)] = [estimate.value, ci[0], ci[1]]
        print([cat,hr,mp], end='')

        ce = estimate.estimate_conditional_effects(mp_cols, 10)
        ce.name = 'ate'
        ce.to_excel(f"{output_directory}/estimate_conditional_effects_q10/{cat}_{hr}_{mp}.xlsx")

        result_df.to_excel(f"{output_directory}/estimates_q10.xlsx")
        # clear_output(wait=True)

        # display(result_df)
        # STOP

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

['is_medical', 15, 30]

['is_medical', 16, 30]

['is_medical', 17, 30]

...

['is_medical', 20, 1]

['is_medical', 21, 1]

['is_medical', 22, 1]

['is_medical', 23, 1]

['is_medical', 24, 1]

In [ ]:
# !ls drive/MyDrive/datathon23/outputs//estimate_conditional_effects -alh -S

In [ ]:
# result_df = pd.concat([pd.read_excel(f"{output_directory}/estimates.xlsx", index_col=[0,1,2,]), pd.read_excel(f"{output_directory}/estimates2.xlsx", index_col=[0,1,2,])])
# result_df.to_excel(f"{output_directory}/estimates.xlsx")

In [ ]:
STOP

In [4]:
categories = ["is_surgical","is_medical"]
# categories = ["is_surgical"]
mp_categories = list(range(5,41))
hr_categories = list(range(1,25))

print(categories)
print(hr_categories)
print(mp_categories)

['is_surgical', 'is_medical']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]


In [5]:
import ast

import pandas as pd
from tqdm.notebook import tqdm as tqdm

def interval_type(s):
    """Parse interval string to Interval"""

    table = str.maketrans({'[': '(', ']': ')'})
    left_closed = s.startswith('[')
    right_closed = s.endswith(']')

    left, right = ast.literal_eval(s.translate(table))

    t = 'neither'
    if left_closed and right_closed:
        t = 'both'
    elif left_closed:
        t = 'left'
    elif right_closed:
        t = 'right'

    return pd.Interval(left, right, closed=t)

def convert_index(cond):
  res = cond.copy()
  res.index =res.reset_index().iloc[:,:-1].applymap(interval_type)
  # pd.MultiIndex.from_tuples(cond.reset_index().iloc[:,:-1].applymap(interval_type).values)
  # cond.reset_index().iloc[:,:-1].applymap(interval_type)
  res.index = pd.MultiIndex.from_tuples(res.index)
  res.index.names = cond.index.names
  return res

In [6]:
# df_expand = pd.read_csv(f'{output_directory}/expand.csv')

In [7]:
# df_expand_idx = df_expand.set_index(['mp','peep', 'rr', 'insp_p']).sort_index()
# df_expand_idx.head()

In [8]:
mp_r = list(range(5, 41))
rr_r = list(range(10,41))
peep_r = list(range(5,21))
insp_p_r = list(range(5,31))


tidal_min = 250
tidal_max = 750

In [ ]:
# results = dict()
df_all_result = []
result_df = pd.read_excel(f"{output_directory}/estimates.xlsx", index_col=[0,1,2,]).sort_index()

# result_df = pd.DataFrame([], columns=['cat','hr','mp','value','ci_lower', 'ci_upper'])
# result_df = result_df.set_index(['cat','hr','mp',])

for cat in categories:
  for hr in tqdm(hr_categories):
    # for mp in mp_categories:
    for mp in tqdm(mp_r,leave=False):
      idx = (cat,hr,mp)
      if idx in result_df.index:
        # print(idx)
        cond = pd.read_excel(f"{output_directory}/estimate_conditional_effects/{cat}_{hr}_{mp}.xlsx", index_col=[0,1,2,3])
        cond.index.names = [x.replace('__categorical__', '') for x in cond.index.names]
        cond = convert_index(cond)
        for peep in peep_r:
          for rr in rr_r:
            for insp_p in insp_p_r:
              tidal = mp / (0.098 / 1000 * rr * (peep + insp_p))
              if tidal_min<=tidal<=tidal_max:
                # ate = cond3.loc[tidal]['ate']
                try:
                  ate = cond.loc[peep, rr, insp_p, tidal]['ate']
                  platue = 30
                  delta_p = platue-peep

                  tidal = mp / (0.098 / 1000 * rr * (peep + insp_p))
                  df_all_result.append((cat,hr,	mp, rr,	insp_p,	peep,	platue,	delta_p, tidal, ate))
                except:
                  pass





        # for t in df.itertuples(index=False):
        #   # peep	a2_resp_rate	insp_p	tidal
        #   try:
        #     ate = cond.loc[t.peep, t.rr, t.insp_p, t.tidal]['ate']
        #     df_all_result.append([cat,hr,] + list(t) + [ate])
        #   except:
        #     pass
        #   # break


        # STOP
    # break
cond
df_all_result = pd.DataFrame(df_all_result, columns = ('cat','hr',	'mp', 'rr',	'insp_p',	'peep',	'platue',	'delta_p', 'tidal', 'ate'))

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

In [ ]:
df_all_result.head()
df_all_result.shape
df_all_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4041360 entries, 0 to 4041359
Data columns (total 10 columns):
 #   Column   Dtype  
---  ------   -----  
 0   cat      object 
 1   hr       int64  
 2   mp       int64  
 3   rr       int64  
 4   insp_p   int64  
 5   peep     int64  
 6   platue   int64  
 7   delta_p  int64  
 8   tidal    float64
 9   ate      float64
dtypes: float64(2), int64(7), object(1)
memory usage: 308.3+ MB


In [ ]:
# df_all_result.to_excel(f"{output_directory}/df_all_result.xlsx")
# df_all_result.head(10_000).to_csv(f"{output_directory}/df_all_result_10000.csv", index=False)

df_all_result.to_csv(f"{output_directory}/df_all_result1.csv", index=False)
